In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import plotly.express as px

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
import nltk
nltk.download("vader_lexicon")
import webbrowser
import os

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mujah\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
apps_df=pd.read_csv('Play Store Data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Play Store Data.csv'

In [ ]:
apps_df=apps_df.dropna(subset=["Rating"])
for column in  apps_df.columns:
   apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df[apps_df['Rating'] <= 5]

In [ ]:
apps_df.reset_index

In [ ]:
#Convert the Installs column to numeric(int) by removing commas and +
apps_df['Installs'] = apps_df['Installs'].str.replace(',', '').str.replace('+', '').astype(int)
#converting Price column after removing $
apps_df['Price'] = apps_df['Price'].str.replace('$','').astype(float)
apps_df['Log_Installs']=np.log(apps_df['Installs'])
apps_df['Reviews']=apps_df['Reviews'].astype(int)
apps_df['Log_Reviews']=np.log(apps_df['Reviews'])
apps_df['Revenue']=apps_df['Price']*apps_df['Installs']
apps_df['Last Updated']=pd.to_datetime(apps_df['Last Updated'],errors='coerce')
apps_df['year']=apps_df['Last Updated'].dt.year

In [ ]:
def rating_group(rating):
    if rating>=4:
        return('Top rated app')
    elif rating>=3:
        return('Above average')
    elif rating>=2:
        return('Average')
    else :
        return('Below Average')
apps_df['Rating_Group']=apps_df['Rating'].apply(rating_group)
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M',''))
    elif 'K' in size:
        return float(size.replace('K',''))/1024
    else:
         return np.nan
apps_df['Size']=apps_df['Size'].apply(convert_size)

In [ ]:
reviews_df=pd.read_csv('User Reviews.csv')

In [ ]:
reviews_df.dropna(subset=['Translated_Review'],inplace=True)

In [ ]:
merged_df=pd.merge(apps_df,reviews_df,on='App',how='inner')
merged_df

In [ ]:
merged_df.to_csv('merged_df.csv',index=False)

In [ ]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [ ]:
plot_containers=""

In [ ]:
def save_plot_as_html(fig,filename,insight):
    global plot_containers
    filepath=os.path.join(html_files_path,filename)
    html_content=pio.to_html(fig,full_html=False,include_plotlyjs='inline')
    plot_containers +=f"""
    <div class = "plot-container" id="{filename}" onclick ="openPlot('{filename}')">
        <div class="plot">{html_content}</div> 
        <div class="insights">{insight}</div>
       </div>
        """
    fig.write_html(filepath,full_html=False,include_plotlyjs='inline')

In [ ]:
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [ ]:
plot_containers_split = plot_containers.split('</div')

In [ ]:
if len(plot_containers_split)>1:
    final_plot=plot_containers_split[-2]
else:
    final_plot=plot_containers

In [ ]:
dashboard_html="""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset = "UTF-8">
  <meta name="viewport"content ="width=device-width,initial-scale=1.0">
  <title> Google Play Store Review Analytics</title>
  <style>
     body{{
        font-family:Arial,sans-serif;
        background-color:#333;
        color:#fff;
        margin:0;
        padding:0;
   }}
   .header {{
   display:flex;
   align-items:center;
   justify-content:center;
   padding:20px;
   background-color:#444
   }}
   .header img {{
      margin: 0 10px;
      height:50px;
   }}
   .container{{
   display:flex;
   flex-wrap:wrap;
   justify-content:center;
   padding:20px;
   }}
   .plot_container{{
     border:2px solid#555;
     margin:10px;
     padding:10px;
     width:{plot_width}px;
     height:{plot_height}px;
     overflow:hidden;
     position:relative;
     cursor:pointer;
     transition:all ease-in-out 0.5s;
   }}
   .insights{{
     display:none;
     position:absolute;
     right:10px;
     top:10px;
     background-color:rgba(0,0,0,0.7);
     padding:5px;
     border-radius:5px;
     color:#fff;
     
   }}
   .plot_container: hover .insights{{
     display:block;
      }}
   </style>
   <script>
   function openPlot(filename){{
      window.open(filename,'_blank');
   }}
  
   </script>
</head> 
<body>
     <div class="header">
    <img src="https://upload.wikimedia.org/wikipedia/commons/4/4a/Logo_2013_Google.png" alt="Google Logo">
    <h1>Google Play Store Reviews Analytics</h1>
     <img src="https://upload.wikimedia.org/wikipedia/commons/7/78/Google_Play_Store_badge_EN.svg" alt="Google Play Store Logo">
</div>
<div class="container">
     {plot}
    </div>
</body>
</html>
"""

In [ ]:
final_html=dashboard_html.format(plot=plot_containers,plot_width=plot_width,plot_height=plot_height)

In [ ]:
dashboard_path=os.path.join(html_files_path,"Web page.html")
with open (dashboard_path,"w",encoding="utf-8")as f:
    f.write(final_html)
webbrowser.open('file://' +os.path.realpath(dashboard_path))

In [ ]:
#Task1:Create a scatter plot to visualize the relationship between revenue and the number of installs for paid apps only.Add a trendline to show the correlation and color-code the points based on app categories.

#  Filter only paid apps
paid_apps = apps_df[apps_df['Type'] == 'Paid'].copy()

paid_apps['Revenue'] = paid_apps['Price'] * paid_apps['Installs']

#  Create scatter plot with trendline
fig1= px.scatter(
    paid_apps,
    x='Installs', 
    y='Revenue',
    color='Category',  # Color-code by category
    title='Revenue vs Installs for Paid Apps',
    labels={'Installs': 'Number of Installs', 'Revenue': 'Revenue ($)'},
    trendline="ols", 
    log_x=True,  # Log scale to spread out low values
    log_y=True,
    width=800,
    height=500
)

#  Improve layout for readability
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),         
    margin=dict(l=10,r=10,t=30,b=10)
)

fig1.show()
save_plot_as_html(fig1,"Graph1.html","Revenue Vs Installs for Paid Apps")



In [ ]:
#Task2:Visualize the sentiment distribution (positive, neutral, negative) of user reviews using a stacked bar chart, segmented by rating groups (e.g., 1-2 stars, 3-4 stars, 4-5 stars). Include only apps with more than 1,000 reviews and group by the top 5 categories.
#task2

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Filter apps with more than 1,000 reviews
filtered_apps = apps_df[apps_df['Reviews'].astype(int) > 1000].copy()

# Select top 5 categories based on count
top_5_categories = filtered_apps['Category'].value_counts().nlargest(5).index
filtered_apps = filtered_apps[filtered_apps['Category'].isin(top_5_categories)]

# Function to get sentiment
def get_sentiment(text):
    if pd.isna(text) or not isinstance(text, str):  # Handle missing/non-string values
        return "Neutral"
    score = sia.polarity_scores(text)["compound"]  # Get sentiment score
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the 'Review' column
filtered_apps["Sentiment"] = filtered_apps["Reviews"].apply(get_sentiment)

# Create rating groups (1-2 stars, 3-4 stars, 4-5 stars)
filtered_apps["Rating Group"] = pd.cut(
    filtered_apps["Rating"],
    bins=[0, 2, 4,5 ],  # Rating bins: (0-2], (2-4], (4-5]
    labels=["1-2 Stars", "3-4 Stars", "4-5 Stars"]
)

# Aggregate sentiment distribution within each category & rating group
sentiment_distribution = (
    filtered_apps.groupby(["Category", "Rating Group", "Sentiment"])
    .size()
    .reset_index(name="Count")
)

# Create a stacked bar chart
fig2 = px.bar(
    sentiment_distribution,
    x="Rating Group",
    y="Count",
    color="Sentiment",
    facet_col="Category",
    title="Sentiment Distribution by Rating Group (Top 5 Categories)",
    labels={"Rating Group": "Rating Group", "Count": "Number of Reviews"},
    color_discrete_map={"Positive": "green", "Neutral": "gray", "Negative": "red"},
    width=900,
    height=500
)

# Dark mode layout adjustments
fig2.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size": 16},
    xaxis=dict(title_font={"size": 12}),
    yaxis=dict(title_font={"size": 12}),
    margin=dict(l=10, r=10, t=50, b=10)
)

fig2.show()
save_plot_as_html(fig2,"Graph2.html","Sentiment Distribution by Rating Group")


In [ ]:
#3.Create a dual-axis chart comparing the average installs and revenue for free vs. paid apps within the top 3 app categories. Apply filters to exclude apps with fewer than 10,000 installs and revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and content rating should be Everyone and app name should not have more than 30 characters including space and special character .this graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.
#task3

# Convert 'Installs' and 'Price' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)
apps_df['Price'] = apps_df['Price'].astype(str).replace(r'\$', '', regex=True).astype(float)

# Calculate revenue for paid apps
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

# Ensure 'Android Ver' is numeric (extract major version)
apps_df['Android Ver'] = apps_df['Android Ver'].astype(str).str.extract(r'(\d+\.\d+)').astype(float)

# Convert 'Size' to numeric (handling MB & KB values)
def convert_size(size):
    if isinstance(size, str):  # Ensure only strings are processed
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'K' in size:
            return float(size.replace('K', '')) / 1024  # Convert KB to MB
    return None  # Return None for missing values

apps_df['Size'] = apps_df['Size'].apply(convert_size)

# Filter data based on conditions
filtered_apps = apps_df[
    (apps_df['Installs'] >= 10000) &  # At least 10,000 installs
    (apps_df['Revenue'] >= 10000) &  # At least $10,000 revenue
    (apps_df['Android Ver'] > 4.0) &  # Android version > 4.0
    (apps_df['Size'] > 15) &  # Size > 15MB
    (apps_df['Content Rating'] == 'Everyone') &  # Only 'Everyone' rating
    (apps_df['App'].apply(lambda x: isinstance(x, str) and len(x) <= 30))  # App name ≤30 characters
]

# Select the top 3 categories based on the number of apps
top_3_categories = filtered_apps['Category'].value_counts().nlargest(3).index
filtered_apps = filtered_apps[filtered_apps['Category'].isin(top_3_categories)]

# Compute average installs & revenue for Free vs Paid apps
category_summary = (
    filtered_apps.groupby(['Category', 'Type'])[['Installs', 'Revenue']]
    .mean()
    .reset_index()
)

# Reshape data for Plotly Express
category_summary_melted = category_summary.melt(
    id_vars=['Category', 'Type'],
    value_vars=['Installs', 'Revenue'],
    var_name='Metric',
    value_name='Value'
)

# Create a grouped bar chart for Installs and Revenue
fig3 = px.bar(
    category_summary_melted,
    x="Category",
    y="Value",
    color="Metric",
    barmode="group",
    facet_col="Type",
    title="Comparison of Average Installs & Revenue (Free vs Paid)",
    labels={"Category": "App Category", "Value": "Average Value", "Metric": "Measure"},
    color_discrete_map={"Installs": "blue", "Revenue": "red"},
    width=900,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=13, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=14, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 1 PM - 2 PM IST
if start_time <= current_time_ist <= end_time:
    fig1.show()
else:
    print("Graph is not available outside 1 PM - 2 PM IST")

fig3.show()

save_plot_as_html(fig3,"Graph3.html","Comparision of Average Installs & Revenue(Free vs Paid")


In [ ]:
#Task4. Create an interactive Choropleth map using Plotly to visualize global installs by Category. Apply filters to show data for only the top 5 app categories and highlight category where the number of installs exceeds 1 million. The app category should not start with the characters “A,” “C,” “G,” or “S.” This graph should work only between 6 PM IST and 8 PM IST; apart from that time, we should not show it in the dashboard itself.
#task4

# Convert 'Installs' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)

# Assign random countries if the dataset lacks country information
country_list = ["United States", "India", "United Kingdom", "Germany", "Canada", "France", "Brazil", "Australia"]
apps_df['Country'] = [random.choice(country_list) for _ in range(len(apps_df))]

# Filter out categories that start with "A", "C", "G", or "S"
apps_df = apps_df[~apps_df['Category'].str.startswith(('A', 'C', 'G', 'S'))]

# Select the top 5 categories based on the number of apps
top_5_categories = apps_df['Category'].value_counts().nlargest(5).index
filtered_apps = apps_df[apps_df['Category'].isin(top_5_categories)]

# Aggregate installs by country and category
category_installs = filtered_apps.groupby(['Category', 'Country'])['Installs'].sum().reset_index()

# Add a highlight column for categories where installs exceed 1 million
category_installs['Highlight'] = category_installs['Installs'].apply(lambda x: 'Highlighted' if x > 1_000_000 else 'Normal')

# Create a choropleth map
fig4= px.choropleth(
    category_installs,
    locations='Country',
    locationmode='country names',
    color='Installs',
    hover_name='Category',
    title="Global Installs by App Category",
    color_continuous_scale='Blues',
    labels={'Installs': 'Total Installs'},
    width=900,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=18, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=20, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 6 PM - 8 PM IST
if start_time <= current_time_ist <= end_time:
    fig4.show()
else:
    print("Graph is not available outside 6 PM - 8 PM IST")

save_plot_as_html(fig4,"Graph4.html","Global Installs by Catregory")


In [ ]:
#Task5.Use a grouped bar chart to compare the average rating and total review count for the top 10 app categories by number of installs. Filter out any categories where the average rating is below 4.0 and size below 10 M and last update should be Jan month . this graph should work only between 3PM IST to 5 PM IST apart from that time we should not show this graph in dashboard itself.
#task5

# Convert 'Installs' and 'Reviews' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)
apps_df['Reviews'] = apps_df['Reviews'].astype(str).str.replace(',', '', regex=True).astype(float)

# Ensure 'Rating' is numeric
apps_df['Rating'] = pd.to_numeric(apps_df['Rating'], errors='coerce')

# Convert 'Size' to numeric (handling MB & KB values)
def convert_size(size):
    if isinstance(size, str):  # Ensure only strings are processed
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'K' in size:
            return float(size.replace('K', '')) / 1024  # Convert KB to MB
    return None  # Return None for missing values

apps_df['Size'] = apps_df['Size'].apply(convert_size)

# Convert 'Last Updated' to datetime format
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')

# Filter data based on conditions
filtered_apps = apps_df[
    (apps_df['Rating'] >= 4.0) &  # Rating must be 4.0 or above
    (apps_df['Size'] >= 10) &  # Size must be at least 10 MB
    (apps_df['Last Updated'].dt.month == 1)  # Must be updated in January
]

# Select the top 10 categories based on total installs
top_10_categories = filtered_apps.groupby('Category')['Installs'].sum().nlargest(10).index
filtered_apps = filtered_apps[filtered_apps['Category'].isin(top_10_categories)]

# Compute average rating & total reviews per category
category_summary = (
    filtered_apps.groupby('Category')
    .agg({'Rating': 'mean', 'Reviews': 'sum'})
    .reset_index()
)

# Reshape data for Plotly Express
category_summary_melted = category_summary.melt(
    id_vars=['Category'],
    value_vars=['Rating', 'Reviews'],
    var_name='Metric',
    value_name='Value'
)

# Create a grouped bar chart
fig5 = px.bar(
    category_summary_melted,
    x="Category",
    y="Value",
    color="Metric",
    barmode="group",
    title="Comparison of Average Rating & Total Reviews (Top 10 Categories)",
    labels={"Category": "App Category", "Value": "Value", "Metric": "Measure"},
    color_discrete_map={"Rating": "blue", "Reviews": "red"},
    width=900,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=15, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=17, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 3 PM - 5 PM IST
if start_time <= current_time_ist <= end_time:
    fig5.show()
else:
    print("Graph is not available outside 3 PM - 5 PM IST")

save_plot_as_html(fig5,"Graph5.html","svcadbcdan")

In [ ]:
#Task6.Plot a time series line chart to show the trend of total installs over time, segmented by app category. Highlight periods of significant growth by shading the areas under the curve where the increase in installs exceeds 20% month-over-month and content rating should be teen and app name should start with letter ‘E’ and installs should be more than 10k as well as this graph should work only between 6 PM IST to 9 PM IST apart from that time we should not show this graph in dashboard itself.
#task6

# Convert 'Installs' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)

# Convert 'Last Updated' to datetime
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')

# Extract Year-Month for aggregation
apps_df['Year-Month'] = apps_df['Last Updated'].dt.to_period('M')

# Filter based on conditions
filtered_apps = apps_df[
    (apps_df['Content Rating'] == 'Teen') &  # Only Teen-rated apps
    (apps_df['App'].str.startswith('E', na=False)) &  # App name starts with 'E'
    (apps_df['Installs'] > 10_000)  # More than 10k installs
]

# Group by category & month to calculate total installs
install_trends = (
    filtered_apps.groupby(['Category', 'Year-Month'])
    .agg({'Installs': 'sum'})
    .reset_index()
)

# Convert 'Year-Month' to datetime format for plotting
install_trends['Year-Month'] = install_trends['Year-Month'].astype(str)
install_trends['Year-Month'] = pd.to_datetime(install_trends['Year-Month'])

# Compute month-over-month growth
install_trends['MoM Growth'] = install_trends.groupby('Category')['Installs'].pct_change() * 100

# Create a time series line chart
fig6 = px.line(
    install_trends,
    x='Year-Month',
    y='Installs',
    color='Category',
    title="Trend of Total Installs Over Time (Segmented by Category)",
    labels={'Year-Month': 'Time (Year-Month)', 'Installs': 'Total Installs'},
    width=900,
    height=500
)

# Highlight areas where month-over-month growth > 20%
highlighted_areas = install_trends[install_trends['MoM Growth'] > 20]

fig6.add_trace(
    px.area(
        highlighted_areas,
        x='Year-Month',
        y='Installs',
        color='Category'
    ).data[0]
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=18, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=21, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 6 PM - 9 PM IST
if start_time <= current_time_ist <= end_time:
    fig.show()
else:
    print("Graph is not available outside 6 PM - 9 PM IST")

save_plot_as_html(fig6,"Graph6.html","hdbcdakdnc")


In [ ]:
#Task7.Create a violin plot to visualize the distribution of ratings for each app category, but only include categories with more than 50 apps and app name should contain letter “C” and exclude apps with fewer than 10 reviews and rating should be less 4.0. this graph should work only between 4 PM IST to 6 PM IST apart from that time we should not show this graph in dashboard itself.
#task7
# Convert 'Reviews' to numeric
apps_df['Reviews'] = apps_df['Reviews'].astype(str).str.replace(',', '', regex=True).astype(float)

# Ensure 'Rating' is numeric
apps_df['Rating'] = pd.to_numeric(apps_df['Rating'], errors='coerce')

# Filter data based on conditions
filtered_apps = apps_df[
    (apps_df['Rating'] < 4.0) &  # Rating must be below 4.0
    (apps_df['Reviews'] >= 10) &  # Must have at least 10 reviews
    (apps_df['App'].str.contains('C', case=False, na=False))  # App name contains 'C'
]

# Select categories with more than 50 apps
category_counts = filtered_apps['Category'].value_counts()
valid_categories = category_counts[category_counts > 50].index
filtered_apps = filtered_apps[filtered_apps['Category'].isin(valid_categories)]

# Create violin plot
fig7 = px.violin(
    filtered_apps,
    x='Category',
    y='Rating',
    box=True,  # Show box plot inside violin
    points='all',  # Show all points
    title="Distribution of Ratings by App Category",
    labels={'Category': 'App Category', 'Rating': 'Rating'},
    width=900,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=16, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=18, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 4 PM - 6 PM IST
if start_time <= current_time_ist <= end_time:
    fig7.show()
else:
    print("Graph is not available outside 4 PM - 6 PM IST")

save_plot_as_html(fig7,"Graph7.html" ,"jdbubdcw")

In [ ]:
#Task8.Plot a bubble chart to analyze the relationship between app size (in MB) and average rating, with the bubble size representing the number of installs. Include a filter to show only apps with a rating higher than 3.5 and that belong to the "Games" category and installs should be more than 50k as well as this graph should work only between 5 PM IST to 7 PM IST apart from that time we should not show this graph in dashboard itself.
#task8
import pandas as pd
import plotly.express as px
import datetime

# Convert 'Installs' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)

# Ensure 'Rating' is numeric
apps_df['Rating'] = pd.to_numeric(apps_df['Rating'], errors='coerce')

# Convert 'Size' to numeric (handling MB & KB values)
def convert_size(size):
    if isinstance(size, str):  # Ensure only strings are processed
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'K' in size:
            return float(size.replace('K', '')) / 1024  # Convert KB to MB
    return None  # Return None for missing values

apps_df['Size'] = apps_df['Size'].apply(convert_size)

# Filter data based on conditions
filtered_apps = apps_df[
    (apps_df['Rating'] > 3.5) &  # Rating must be greater than 3.5
    (apps_df['Category'] == 'Games') &  # Category must be "Games"
    (apps_df['Installs'] > 50_000)  # More than 50,000 installs
]

# Create a bubble chart
fig8= px.scatter(
    filtered_apps,
    x='Size',
    y='Rating',
    size='Installs',
    color='Installs',
    title="App Size vs. Average Rating (Bubble Size = Installs)",
    labels={'Size': 'App Size (MB)', 'Rating': 'Average Rating', 'Installs': 'Number of Installs'},
    color_continuous_scale='Viridis',
    width=900,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=17, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=19, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 5 PM - 7 PM IST
if start_time <= current_time_ist <= end_time:
    fig8.show()
else:
    print("Graph is not available outside 5 PM - 7 PM IST")

save_plot_as_html(fig8,"Graph8.html","lsjnfvdsv")


In [ ]:
#Task9.Generate a heatmap to show the correlation matrix between installs, ratings, and review counts. Filter the data to include only apps that have been updated within the last year and have at least 100,000 installs and reviews count should be more than 1k and genres name should not be Starting with characters A , F , E , G , I , K . this graph should work only between 2 PM IST to 4 PM IST apart from that time we should not show this graph in dashboard itself.
#task9

# Convert 'Installs' and 'Reviews' to numeric
apps_df['Installs'] = apps_df['Installs'].astype(str).str.replace(',', '', regex=True).str.replace(r'\+', '', regex=True).astype(float)
apps_df['Reviews'] = apps_df['Reviews'].astype(str).str.replace(',', '', regex=True).astype(float)

# Ensure 'Rating' is numeric
apps_df['Rating'] = pd.to_numeric(apps_df['Rating'], errors='coerce')

# Convert 'Last Updated' to datetime
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')

# Define one year ago from today
one_year_ago = pd.Timestamp.today() - pd.DateOffset(years=1)

# Filter data based on conditions
filtered_apps = apps_df[
    (apps_df['Last Updated'] >= one_year_ago) &  # Updated within the last year
    (apps_df['Installs'] >= 100_000) &  # At least 100,000 installs
    (apps_df['Reviews'] > 1_000) &  # More than 1,000 reviews
    (~apps_df['Genres'].str.startswith(('A', 'F', 'E', 'G', 'I', 'K'), na=False))  # Exclude genres with these starting letters
]

# Compute correlation matrix
correlation_matrix = filtered_apps[['Installs', 'Rating', 'Reviews']].corr()

# Convert correlation matrix to long format
correlation_long = correlation_matrix.stack().reset_index()
correlation_long.columns = ['Variable 1', 'Variable 2', 'Correlation']

# Create heatmap
fig9 = px.imshow(
    correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    color_continuous_scale='RdBu_r',
    title="Correlation Heatmap (Installs, Ratings, Reviews)",
    width=700,
    height=500
)

# Get the current time in IST
current_time_ist = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(hours=5, minutes=30)
start_time = current_time_ist.replace(hour=14, minute=0, second=0, microsecond=0)
end_time = current_time_ist.replace(hour=16, minute=0, second=0, microsecond=0)

# Ensure the graph is only shown between 2 PM - 4 PM IST
if start_time <= current_time_ist <= end_time:
    fig9.show()
else:
    print("Graph is not available outside 2 PM - 4 PM IST")

save_plot_as_html(fig9,"Graph9.html","diwdopkwdk")
